# 模型合并

Lora微调后只会保存微调的部分参数，此处通过加载原始模型和Lora部分结合后存为新的模型，之后即可直接加载新模型进行使用

In [6]:
from transformers import AutoModelForCausalLM
from peft import PeftModel
import os

In [3]:
# 加载Qwen原模型
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")

# 转为peft模型
lora_weight = os.path.join(os.getcwd(), "weight", "checkpoint-500")
model = PeftModel.from_pretrained(model, model_id=lora_weight)

In [5]:
# 合并模型
save_path = os.path.join(os.getcwd(), "weight", "merge_weight")
merge_model = model.merge_and_unload()
merge_model.save_pretrained(save_path)

# 模型测试

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import os 
import torch

weight_path = os.path.join(os.getcwd(), "weight", "merge_weight")
model = AutoModelForCausalLM.from_pretrained(weight_path)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
model.to("cuda")
model.device

device(type='cuda', index=0)

In [17]:
instruction = "你是文本分类领域的专家，我将会给你一段文本和几个潜在的文本分类选项，请根据文本内容输出正确的文本类型。"
input = "文本:航空动力学报JOURNAL OF AEROSPACE POWER1998年 第4期 No.4 1998科技期刊管路系统敷设的并行工程模型研究*陈志英*　*　马　枚北京航空航天大学【摘要】　提出了一种应用于并行工程模型转换研究的标号法，该法是将现行串行设计过程(As-is)转换为并行设计过程(To-be)。本文应用该法将发动机外部管路系统敷设过程模型进行了串并行转换，应用并行工程过程重构的手段，得到了管路敷设并行过程模型。"
category = ""

message = [
    {"role": "system", "content": instruction},
    {"role": "user", "content": input}
]

In [25]:
def predict(message):
    prompt = tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([prompt], return_tensors="pt")
    model_inputs = {k: v.to(model.device) for k, v in model_inputs.items()}

    with torch.inference_mode():
        output = model.generate(**model_inputs, max_new_tokens=512)
        print(output)
        result = tokenizer.decode(output[0, model_inputs["input_ids"].shape[1]:], skip_special_tokens=True)
    
    return result

In [26]:
predict(message)

tensor([[151644,   8948,    198, 105043, 108704,  70538, 104799, 101057,   3837,
          35946, 104347, 104314, 104383, 108704,  33108, 100204, 106362,   9370,
         108704,  70538, 109487,  37945, 100345, 108704,  43815,  66017, 105045,
         108704,  31905,   1773, 151645,    198, 151644,    872,    198, 108704,
             25, 101276, 100738,  47764,  42278,     41,  94699,   3008,    362,
            640,   3126,  23383,  42637,     16,     24,     24,     23,   7948,
          50331,     19,  22704,   2308,     13,     19,    220,     16,     24,
             24,     23,  99602, 109862,  35551,  45995,  72448, 107659,  29635,
           9370,  62926,  22243,  99655, 104949,  99556,      9, 100348,  77128,
          82847,      9,  22441,      9,  22441,  99313,  22441, 102991,  68990,
         117792,  99562,  10904, 106208,  10958,  22441, 101080,  99593,  86402,
         110645,  62926,  22243,  99655, 104949, 105359,  99556,   9370,  30844,
          17992,  24339,   3

'科技'